# Menjalankan BigQuery Python di Jupyter Notebook

BigQuery adalah gudang data ( data warehouse ) yang tidak memiliki server ( serverless ), dapat mengkueri data dalam skala petabyte, dirancang untuk membuat semua analis data anda produktif dengan kinerja tinggi dengan harga yang relatif murah. Karena tidak ada infrastruktur untuk dikelola, jadi kita dapat fokus menganalisis data untuk menemukan wawasan yang bermakna menggunakan SQL tanpa perlu administrator database.

BigQuery memungkinkan anda untuk menganalisis data secara realtime menggunakan kemampuan streaming data sehingga menghasilkan wawasan selalu terkini dan gratis hingga 1 TB data untuk kueri setiap bulan dan 10 GB data yang disimpan.

Untuk menggunakan BigQuery bagi permula, Google menawarkan 2 opsi yaitu menggunakan Sandbox dan Gratis uji coba. Dalam tutorial ini kita akan menggunakan versi Sandbox. Jika anda sudah mendaftar versi uji coba atau bahkan berbayar juga dapat mengikuti tutorial ini.

## Prasyarat

1. Mendaftar BigQuery https://cloud.google.com/bigquery/
2. Aktifkan [BigQuery API](https://console.cloud.google.com/flows/enableapi?apiid=bigquery&_ga=2.260443277.-1046273168.1553369088)

Daftar Isi Konten :

1. [Install Library ](#1.-Install-library)
2. [Otentikasi menggunakan Service Account](#2.-Otentikasi-menggunakan-Service-Account)
3. [Mengelola Datasets](#3.-Mengelola-dataset)
4. [Mengelola Tabel](#4.-Mengelola-tabel)
5. [Menjalankan Kueri Data](#5.-Menjalankan-Kueri-Data)
6. [Memanage Jobs](#6.-Memanage-Jobs)

<hr />

## 1. Install library

In [4]:
#!pip install --upgrade google-cloud-bigquery

## 2. Otentikasi menggunakan Service Account

Menggunakan kredensial Service Account sangat berguna saat bekerja pada remote server tanpa akses ke input pengguna.

* Masuk ke https://console.cloud.google.com/apis/credentials/serviceaccountkey

<img width="auto" height="auto" src="https://storage.googleapis.com/tommypratama/imgcdn/Screen%20Shot%202019-03-25%20at%2002.58.55.png">

* Kemudian pada **Service account** pilih **New service account**
* **Service account name** : isi dengan nama yang unik
* Pada bagian **Role** pilih **BigQuery > BigQuery Admin**
* **Key type** menggunakan format JSON
* Klik tombol **Create** dan simpan file
* Ganti nama file menjadi **key.json** ( Ini hanya opsi untuk memudahkan melakukan load file saja, jika tidak dirubah juga tidak masalah) .

## Import modul bigquery dan setting kredensial

In [5]:
# Import modul bigquery
from google.cloud import bigquery

# Jika kredensial tidak di simpan ke dalam variabel client
# maka library client akan mencari kredensial dari project yang terhubung sekarang
## client = bigquery.Client()

# Setting kredensial menggunakan Service Account
client = bigquery.Client.from_service_account_json('key.json')

## 3. Mengelola dataset

### Membuat dataset

In [2]:
# Tentukan project dataset id
dataset_id = 'my_dataset'

# Buat DatasetReference menggunakan ID dataset yang dipilih.
dataset_ref = client.dataset(dataset_id)

# Buat objek Dataset lengkap untuk dikirim ke API.
dataset = bigquery.Dataset(dataset_ref)
# Tentukan lokasi geografis di mana dataset harus berada.
dataset.location = "US"

# Kirim dataset ke API untuk dibuat.
# Meningkatkan google.api_core.exceptions.Conflict jika Dataset sudah siap
# Dan ada dalam project 
dataset = client.create_dataset(dataset)  # API request

Sekarang coba lihat pada halaman https://console.cloud.google.com/bigquery maka akan muncul datasets dengan nama **my_datasets** .


<img align="left" width="700" height="700" src="https://storage.googleapis.com/tommypratama/imgcdn/Screen%20Shot%202019-03-25%20at%2003.34.21.png">

**Atau bisa menggunakan perintah berikut ini untuk melihat daftar datasets yang ada dalam project**

In [3]:
datasets = list(client.list_datasets())
project = client.project

if datasets:
    print('Datasets in project {}:'.format(project))
    for dataset in datasets:  # API request(s)
        print('\t{}'.format(dataset.dataset_id))
else:
    print('{} project does not contain any datasets.'.format(project))

Datasets in project bigquery-sandbox-235419:
	my_dataset


### Melihat informasi dataset

In [4]:
# Nama dataset
dataset_id = 'my_dataset'

dataset_ref = client.dataset(dataset_id)
dataset = client.get_dataset(dataset_ref)  # API request

# Lihat properti dataset
print('Dataset ID: {}'.format(dataset_id))
print('Description: {}'.format(dataset.description))
print('Labels:')
labels = dataset.labels
if labels:
    for label, value in labels.items():
        print('\t{}: {}'.format(label, value))
else:
    print("\tDataset tidak memiliki label yang ditentukan.")

# Lihat tabel di dataset
print('Tables:')
tables = list(client.list_tables(dataset_ref))  # API request(s)
if tables:
    for table in tables:
        print('\t{}'.format(table.table_id))
else:
    print('\tDataset ini tidak memiliki tabel apa pun.')

Dataset ID: my_dataset
Description: None
Labels:
	Dataset has no labels defined.
Tables:
	This dataset does not contain any tables.


## 4. Mengelola tabel

### Buat tabel kosong dengan skema:

In [9]:
dataset_ref = client.dataset('my_dataset')

schema = [
    bigquery.SchemaField('full_name', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('age', 'INTEGER', mode='REQUIRED'),
]
table_ref = dataset_ref.table('my_table')
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table)  # API request

assert table.table_id == 'my_table'

### Masukkan baris ke data tabel 

Menggunakan metode `` insert_rows() `` :

```sql
dataset_id = 'my_dataset'  # replace with your dataset ID
# For this sample, the table must already exist and have a defined schema
table_id = 'my_table'  # replace with your table ID
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)  # API request

rows_to_insert = [(u"Phred Phlyntstone", 32), (u"Wylma Phlyntstone", 29)]

errors = client.insert_rows(table, rows_to_insert)  # API request

assert errors == []
```

> **Catatan** : Untuk melakukan streaming data ( insert ) tidak berlaku untuk **BigQuery Sandbox**

### Membuat tabel dari hasil kueri ke tabel permanen

BigQuery menyimpan semua hasil kueri ke tabel, yang bisa permanen atau sementara:

* Tabel sementara adalah tabel yang diberi nama secara acak yang disimpan dalam dataset khusus. Tabel sementara digunakan untuk [men-cache](https://cloud.google.com/bigquery/docs/cached-results) hasil kueri. Tabel sementara memiliki batas waktu sekitar 24 jam. Tabel sementara tidak tersedia untuk dibagikan, dan tidak terlihat menggunakan daftar standar atau metode manipulasi tabel lainnya. Anda tidak dikenai biaya untuk menyimpan tabel sementara.

* Tabel permanen dapat berupa tabel baru atau yang sudah ada dalam dataset yang anda akses. Jika anda menulis hasil kueri ke tabel baru, Anda akan dikenakan biaya untuk menyimpan data. Saat anda menulis hasil kueri ke tabel permanen, tabel yang anda kueri harus berada di lokasi yang sama dengan dataset yang berisi tabel tujuan.

Saat Anda menulis hasil kueri ke tabel permanen, anda bisa membuat tabel baru, menambahkan hasilnya ke tabel yang ada, atau menimpa tabel yang sudah ada.

Lihat dokumentasi BigQuery untuk informasi lebih lanjut tentang [menulis hasil kueri](https://cloud.google.com/bigquery/docs/writing-results) .

In [11]:
# Tentukan tujuan dataset_id
dataset_id = 'my_dataset'

job_config = bigquery.QueryJobConfig()
# Atur tabel tujuan
table_ref = client.dataset(dataset_id).table("corpus")
job_config.destination = table_ref
sql = """
    SELECT corpus
    FROM `bigquery-public-data.samples.shakespeare`
    GROUP BY corpus;
"""

# Mulai kueri, melewati konfigurasi tambahan.
query_job = client.query(
    sql,
    # Lokasi harus cocok dengan dataset yang dirujuk dalam kueri
    # dan dari tabel tujuan.
    location="US",
    job_config=job_config,
)  # Permintaan API - memulai kueri

query_job.result()  # Menunggu permintaan kueri selesai
print("Query results loaded to table {}".format(table_ref.path))

Query results loaded to table /projects/bigquery-sandbox-235419/datasets/my_dataset/tables/corpus


### Menyimpan hasil kueri data yang besar menggunakan legacy SQL

In [12]:
# Nama dataset
dataset_id = 'my_dataset'

job_config = bigquery.QueryJobConfig()
# Set use_legacy_sql ke True untuk menggunakan sintaks legacy SQL.
job_config.use_legacy_sql = True
# Atur tabel tujuan
table_ref = client.dataset(dataset_id).table('corpus2')
job_config.destination = table_ref
job_config.allow_large_results = True
sql = """
    SELECT corpus
    FROM [bigquery-public-data:samples.shakespeare]
    GROUP BY corpus;
"""
# Mulai kueri, melewati konfigurasi tambahan.
query_job = client.query(
    sql,
    # Lokasi harus cocok dengan dataset yang dirujuk dalam kueri
    # dan dari tabel tujuan.
    location='US',
    job_config=job_config)  # Permintaan API - memulai kueri

query_job.result()  # Menunggu permintaan kueri selesai
print('Query results loaded to table {}'.format(table_ref.path))

Query results loaded to table /projects/bigquery-sandbox-235419/datasets/my_dataset/tables/corpus2


### Mendapatkan informasi tabel

Melihat informasi yang ada pada tabel menggunakan metode `` get_table() `` :

In [14]:
# Nama dataset
dataset_id = 'my_dataset'
# Nama tabel
table_id = 'corpus'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)  # Permintaan API

# Lihat tabel properti
print(table.schema)
print(table.description)
print(table.num_rows)

[SchemaField('corpus', 'STRING', 'NULLABLE', None, ())]
None
42


Jelajahi baris data dalam tabel dengan metode `` list_rows() `` :

In [32]:
dataset_ref = client.dataset("samples", project="bigquery-public-data")
table_ref = dataset_ref.table("shakespeare")
table = client.get_table(table_ref)  # API call

# Muat semua baris dari sebuah tabel
rows = client.list_rows(table)
assert len(list(rows)) == table.num_rows

# Muat 10 baris pertama
rows = client.list_rows(table, max_results=10)
assert len(list(rows)) == 10

# Tentukan bidang yang dipilih untuk membatasi hasil ke kolom tertentu
fields = table.schema[:2]  # first two columns
rows = client.list_rows(table, selected_fields=fields, max_results=10)
assert len(rows.schema) == 2
assert len(list(rows)) == 10

# Gunakan indeks awal untuk memuat bagian tabel
rows = client.list_rows(table, start_index=10, max_results=10)

# Cetak baris data dalam format tabel
format_string = "{!s:<16} " * len(rows.schema)
field_names = [field.name for field in rows.schema]
print(format_string.format(*field_names))  # prints column headers
for row in rows:
    print(format_string.format(*row))  # prints row data

word             word_count       corpus           corpus_date      
C                1                sonnets          0                
L                1                sonnets          0                
imaginary        1                sonnets          0                
H                1                sonnets          0                
relief           1                sonnets          0                
W                1                sonnets          0                
V                1                sonnets          0                
advised          1                sonnets          0                
grey             1                sonnets          0                
X                1                sonnets          0                


### Melihat isi tabel yang ada pada dataset

In [24]:
# Nama dataset
dataset_id = 'my_dataset'

dataset_ref = client.dataset(dataset_id)
dataset = client.get_dataset(dataset_ref)  # Permintaan API

# Lihat tabel dalam dataset
print('Tables:')
tables = list(client.list_tables(dataset_ref))  # Permintaan API(s)
if tables:
    for table in tables:
        print('\t{}'.format(table.table_id))
else:
    print('\tThis dataset does not contain any tables.')

Tables:
	corpus
	corpus2
	my_table


### Upload data ke tabel BigQuery dari file lokal 

Menggunakan metode `` load_table_from_file() `` :

In [33]:
filename = '_datasets/countries.csv'
dataset_id = 'my_dataset'
table_id = 'countries'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(
        source_file,
        table_ref,
        location="US",  # Harus sama dengan lokasi dataset tujuan
        job_config=job_config,
    )  # Permintaan API

job.result()  # Tunggu sampai kueri selesai

print("Loaded {} rows into {}:{}.".format(job.output_rows, dataset_id, table_id))

Loaded 206 rows into my_dataset:countries.


### Upload CSV data dari Google Cloud Storage menggunakan schema auto-detection

Memuat file CSV dari Cloud Storage dengan metode `` load_table_from_uri() `` :

In [39]:
dataset_id = 'my_dataset'
table_id = 'us_states'
uri = 'gs://cloud-samples-data/bigquery/us-states/us-states.csv'

dataset_ref = client.dataset(dataset_id)
job_config = bigquery.LoadJobConfig()
job_config.autodetect = True
job_config.skip_leading_rows = 1
# Format sumber default ke CSV, jadi baris di bawah ini opsional.
job_config.source_format = bigquery.SourceFormat.CSV

load_job = client.load_table_from_uri(
    uri,
    dataset_ref.table(table_id),
    job_config=job_config)  # Permintaan API
print('Starting job {}'.format(load_job.job_id))

load_job.result()  # Tunggu loading data selesai
print('Job finished.')

destination_table = client.get_table(dataset_ref.table(table_id))
print('Loaded {} rows.'.format(destination_table.num_rows))

Starting job a5339517-5b89-4f7d-bfc2-2f4e1105a582
Job finished.
Loaded 50 rows.


### Menyalin tabel

Salin tabel dengan metode `` copy_table() `` :

In [41]:
source_dataset = client.dataset("samples", project="bigquery-public-data")
source_table_ref = source_dataset.table("shakespeare")

dataset_id = 'my_dataset'
table_id = 'shakespeare'
dest_table_ref = client.dataset(dataset_id).table(table_id)

job = client.copy_table(
    source_table_ref,
    dest_table_ref,
    # Lokasi harus cocok dengan tabel sumber dan tujuan.
    location="US",
)  # Permintaan API

job.result()  # Tunggu job sampai selesai

assert job.state == "DONE"
dest_table = client.get_table(dest_table_ref)  # Permintaan API
assert dest_table.num_rows > 0

Salin data tabel ke Google Cloud Storage dengan metode `` extract_table() `` :

> **Catatan** : Untuk dapat mengupload tabel BigQuery ke Google Cloud Storage anda harus menambahkan permission menggunakan email yang digunakan `` Service Account `` pada file `` key.json ``

1. Masuk ke https://console.cloud.google.com/storage/
2. Klik nama bucket, jika anda belum membuat bucket bisa langsung membuatnya dengan **Create bucket**
3. Pilih **Permission** > **Add members**
4. Masukkan email yang ada pada file **key.json** di bagian **Members** ,kemudian pada bagian **Roles** pilih **Storage** > **Storage Admin** > **Add**

<img align="left" width="600" height="600" src="https://storage.googleapis.com/tommypratama/imgcdn/Screen%20Shot%202019-03-25%20at%2005.58.00.png">


In [45]:
# Nama bucket
bucket_name = 'tabel-bigquery'
# Atur project id, dataset id, tabel id
project = "bigquery-public-data"
dataset_id = "samples"
table_id = "shakespeare"

# Destinasi lokasi file penyimpanan Cloud Storage
destination_uri = "gs://{}/{}".format(bucket_name, "shakespeare.csv")
dataset_ref = client.dataset(dataset_id, project=project)
table_ref = dataset_ref.table(table_id)

extract_job = client.extract_table(
    table_ref,
    destination_uri,
    # Lokasi harus sama dengan sumber tabel.
    location="US",
)  # Permintaan API
extract_job.result()  # tunggu job sampai dengan selesai.

print(
    "Exported {}:{}.{} to {}".format(project, dataset_id, table_id, destination_uri)
)

Exported bigquery-public-data:samples.shakespeare to gs://tabel-bigquery/shakespeare.csv


### Menghapus tabel

Hapus tabel dengan metode `` delete_table() `` :

In [25]:
dataset_id = 'my_dataset' # Ganti dengan dataset ID anda.
table_id = 'corpus2'      # Ganti dengan table ID anda.

table_ref = client.dataset(dataset_id).table(table_id)
client.delete_table(table_ref)  # Permintaan API

print('Tabel {}:{} berhasil dihapus.'.format(dataset_id, table_id))

Tabel my_dataset:corpus2 berhasil dihapus.


## 5. Menjalankan Kueri Data

Jalankan kueri sql berikut ini:

In [22]:
query = (
    "SELECT name FROM `bigquery-public-data.usa_names.usa_1910_2013` "
    'WHERE state = "TX" '
    "LIMIT 10"
)
query_job = client.query(
    query,
    # Lokasi harus cocok dengan dataset yang dirujuk dalam kueri.
    location="US",
)  # Permintaan API - memulai kueri

for row in query_job:  # Permintaan API - mengambil hasil
    # Nilai baris dapat diakses dengan nama field atau indeks
    assert row[0] == row.name == row["name"]
    print(row)

Row(('Frances',), {'name': 0})
Row(('Alice',), {'name': 0})
Row(('Beatrice',), {'name': 0})
Row(('Ella',), {'name': 0})
Row(('Gertrude',), {'name': 0})
Row(('Josephine',), {'name': 0})
Row(('Lula',), {'name': 0})
Row(('Blanche',), {'name': 0})
Row(('Marjorie',), {'name': 0})
Row(('Christine',), {'name': 0})


### Jalankan dry run query

**Dry run query** digunakan untuk melihat seberapa besar file yang akan di proses dalam kueri, sekaligus untuk mengontrol biaya ketika menggunakan kueri data. Jadi jika kita tidak tahu seberapa besar file yang akan di kueri sebaik nya menggunakan metode ini.

In [30]:
# from google.cloud import bigquery
# client = bigquery.Client()

job_config = bigquery.QueryJobConfig()
job_config.dry_run = True
job_config.use_query_cache = False
query_job = client.query(
    (
        "SELECT name, COUNT(*) as name_count "
        "FROM `bigquery-public-data.usa_names.usa_1910_2013` "
        "WHERE state = 'WA' "
        "GROUP BY name"
    ),
    # Lokasi harus cocok dengan dataset yang dirujuk dalam kueri.
    location="US",
    job_config=job_config,
)  # Permintaan API

# Permintaan dry run query selesai segera.
assert query_job.state == "DONE"
assert query_job.dry_run

print("This query will process {} bytes.".format(query_job.total_bytes_processed))

This query will process 65935918 bytes.


### Menggunakan hasil query yang di-cache

BigQuery menulis semua hasil kueri ke tabel. Tabel ini secara eksplisit diidentifikasi oleh pengguna (tabel tujuan), atau tabel hasil sementara yang di-cache. Tabel hasil sementara, di-cache dipertahankan per-pengguna, per-proyek. Tidak ada biaya penyimpanan untuk tabel sementara, tetapi jika Anda menulis hasil kueri ke tabel permanen, Anda akan dikenakan biaya untuk menyimpan data.

Semua hasil kueri, termasuk kueri interaktif dan batch, di-cache dalam tabel sementara selama sekitar 24 jam dengan beberapa pengecualian.

Dokumentasi https://cloud.google.com/bigquery/docs/cached-results

In [21]:
job_config = bigquery.QueryJobConfig()
job_config.use_query_cache = True
sql = """
    SELECT corpus
    FROM `bigquery-public-data.samples.shakespeare`
    GROUP BY corpus
    LIMIT 5;
"""
query_job = client.query(
    sql,
    # Lokasi harus cocok dengan dataset yang dirujuk dalam kueri.
    location='US',
    job_config=job_config)  # Permintaan API

# Cetak hasilnya.
for row in query_job:  # Permintaan API - mengambil hasil
    print(row)

Row(('sonnets',), {'corpus': 0})
Row(('various',), {'corpus': 0})
Row(('1kinghenryvi',), {'corpus': 0})
Row(('2kinghenryvi',), {'corpus': 0})
Row(('3kinghenryvi',), {'corpus': 0})


### Jalankan kueri menggunakan nama parameter kueri

Lihat dokumentasi BigQuery untuk informasi lebih lanjut tentang [parameterized queries](https://cloud.google.com/bigquery/docs/parameterized-queries) .

In [18]:
query = """
    SELECT word, word_count
    FROM `bigquery-public-data.samples.shakespeare`
    WHERE corpus = @corpus
    AND word_count >= @min_word_count
    ORDER BY word_count DESC;
"""
query_params = [
    bigquery.ScalarQueryParameter("corpus", "STRING", "romeoandjuliet"),
    bigquery.ScalarQueryParameter("min_word_count", "INT64", 250),
]
job_config = bigquery.QueryJobConfig()
job_config.query_parameters = query_params
query_job = client.query(
    query,
    # Lokasi harus cocok dengan dataset yang dirujuk dalam kueri.
    location="US",
    job_config=job_config,
)  # Permintaan API - memulai kueri

# Cetak hasil kueri
for row in query_job:
    print("{}: \t{}".format(row.word, row.word_count))

assert query_job.state == "DONE"

the: 	614
I: 	577
and: 	490
to: 	486
a: 	407
of: 	367
my: 	314
is: 	307
in: 	291
you: 	271
that: 	270
me: 	263


## 6. Memanage Jobs

Jobs menggambarkan tindakan yang dilakukan pada data dalam tabel BigQuery:

* Load data ke dalam table
* Menjalankan kueri terhadap data dalam satu atau beberapa tabel
* Ekstrak data dari tabel
* Menyalin tabel

In [47]:
import datetime

# Melihat daftar 10 Jobs terakhir
# Hapus parameter max_result untuk melihat daftar jobs dari 6 bulan terakhir.
print("10 jobs terakhir:")
for job in client.list_jobs(max_results=10):  # Permintaan API(s)
    print(job.job_id)

Last 10 jobs:
7921ab17-170f-4421-8a1b-9c11b7353e80
0a25a319-c374-4225-b05e-a2da2bc1cdce
8a49e089-3231-4aed-bc86-b46dbc5a6489
a5339517-5b89-4f7d-bfc2-2f4e1105a582
29ea4f32-d9c6-41be-b7f6-7639db15f1b3
0a6faeea-dfac-43c5-a930-24b23a7ea019
2af00eff-1864-4b2a-9523-b2d5f332cd3f
cdd0be0b-73ac-4fd4-a45f-491e759d72de
159c7dfd-eeec-495e-963b-87c69140afcf


In [48]:
# Berikut ini adalah contoh parameter opsional tambahan:
# Gunakan min_creation_time atau max_creation_time untuk menentukan waktu
print("Jobs yang berjalan dari 10 menit terakhir:")
ten_mins_ago = datetime.datetime.utcnow() - datetime.timedelta(minutes=10)
for job in client.list_jobs(min_creation_time=ten_mins_ago):
    print(job.job_id)

Jobs from the last ten minutes:


In [6]:
# Gunakan all_users untuk untuk melihat jobs yang dijalankan oleh semua pengguna dalam project.
print("10 jobs terakhir yang dijalankan oleh semua pengguna:")
for job in client.list_jobs(max_results=10, all_users=True):
    print("{} dijalankan oleh user: {}".format(job.job_id, job.user_email))

10 jobs terakhir yang dijalankan oleh semua pengguna:
79ed98f6-3566-4f6c-b007-3b4c82c3de62 dijalankan oleh user: bigquery-free@bigquery-sandbox-235419.iam.gserviceaccount.com
bb086bc0-7858-495c-bbae-b2cb6b027065 dijalankan oleh user: bigquery-free@bigquery-sandbox-235419.iam.gserviceaccount.com
cf347992-de7e-47dd-a52f-12cf5898f56d dijalankan oleh user: bigquery-free@bigquery-sandbox-235419.iam.gserviceaccount.com
03f2ed34-e1c3-4bd6-b60b-2a62785412a2 dijalankan oleh user: bigquery-free@bigquery-sandbox-235419.iam.gserviceaccount.com
4e24585e-9c7c-4e5f-88ec-b088b38e5727 dijalankan oleh user: bigquery-free@bigquery-sandbox-235419.iam.gserviceaccount.com
29a865bf-cc26-4cd9-96cc-aa2dc25f0a1c dijalankan oleh user: bigquery-free@bigquery-sandbox-235419.iam.gserviceaccount.com
bde7979f-4e5b-4a0e-bf58-66e41bdbd178 dijalankan oleh user: bigquery-free@bigquery-sandbox-235419.iam.gserviceaccount.com
93db7856-5d05-421a-bf79-c8a2b9b981c0 dijalankan oleh user: bigquery-free@bigquery-sandbox-235419.ia

In [50]:
# Memfilter jobs yang sedang berjalan menggunakan state.filter
print("Jobs yang sedang berjalan:")
for job in client.list_jobs(state_filter="RUNNING"):
    print(job.job_id)

Jobs currently running:


**Sumber** :

* https://googleapis.github.io/google-cloud-python/latest/bigquery/usage/queries.html
* https://googleapis.github.io/google-cloud-python/latest/bigquery/usage/tables.html
* https://googleapis.github.io/google-cloud-python/latest/bigquery/magics.html
* https://pandas-gbq.readthedocs.io/en/latest/howto/authentication.html